# Предсказания победителя в онлайн-игре

Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче, как правило, участвует 10 человек. Матчи формируются из живой очереди, с учётом уровня игры всех игроков. Перед началом игры игроки автоматически разделяются на две команды по пять человек. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника, трон.

Вам нужно построить модель, которая по данным о первых пяти минутах матча будет предсказывать его исход — то есть определять команду-победителя.

Чтобы выполнить это задание, вам необходимо провести ряд исследований, сравнить несколько алгоритмов машинного обучения и проверить эффект от ряда манипуляций с признаками.

К заданию приложены следующие файлы:

final-statement.ipynb и final-statement.html — постановка задачи, описание данных, инструкции по выполнению  
features.zip — архив с обучающей выборкой  
features_test.zip — архив с тестовой выборкой  
data.zip — полный архив с сырыми данными и скриптом для извлечения признаков (этот архив понадобится вам только для участия в kaggle; для выполнения данного задания он не нужен)  
extract_features.py — скрипт, извлекающий признаки из сырых данных

## Предметная область: Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

## Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?

## Набор данных

Набор данных с матчами записан в файле `matches.jsonlines.bz2`.
В каталоге `dictionaries` приведены расшифровки идентификаторов, которые присутствуют в записях матчей.

#### Чтение информации о матчах

Информация о матчах записана в сжатом текстовом файле `matches.jsonlines.bz2`, каждая строчка которого содержит объект в формате [JSON](https://ru.wikipedia.org/wiki/JSON). Запись в формате JSON преобразуется в python-объект при помощи стандартного модуля `json`. Пример чтения матчей:

In [1]:
import json
import bz2

with bz2.BZ2File('./data/matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)
        
        # Обработка матча
        break

#### Описание полей в записи матча

```python
{
    "match_id": 247,            # идентификатор матча
    "start_time": 1430514316,   # дата/время начала матча, unixtime
    "lobby_type": 0,            # тип комнаты, в которой собираются игроки 
                                #   (расшифровка в dictionaries/lobbies.csv)
 
    # стадия выбора героев
    "picks_bans": [
        {
            "order": 0,       # порядковый номер действия
            "is_pick": false, # true если команда выбирает героя, false — если банит
            "team": 1,        # команда, совершающая действие (0 — Radiant, 1 — Dire)
            "hero_id": 95     # герой, связанный с действием 
                              #    (расшифровка в dictionaries/heroes.csv)
        }, 
        ...
    ],

    # информация про каждого игрока, список ровно из 10 элементов
    # игроки с индексами от 0 до 4 — из команды Radiant, от 5 до 9 — Dire
    "players": [ 
        { 
        
            # герой игрока (расшифровка в dictionaries/heroes.csv)
            "hero_id": 67,  

            # временные ряды (отсчеты указаны в поле "times")
            "xp_t": [0, 13, 115, 177, 335, ...],   # опыт
            "gold_t": [0, 99, 243, 343, 499, ...], # золото + стоимость всех купленных вещей (net worth)
            "lh_t": [0, 0, 2, 2, 2, ...],          # количество убитых юнитов (не героев) противника

            # список событий: улучшение способностей героя
            "ability_upgrades": [
                {
                    "time": 51,      # игровое время
                    "level": 1,      # уровень игрока, на котором произошло улучшение
                    "ability": 5334  # способность, которая была улучшена 
                                     # (расшифровка в dictionaries/abilities.csv)
                }, 
                ...
            ],

            # список событий: убийства
            "kills_log": [
                {
                    "time": 831,    # игровое время
                    "player": 7,    # индекс игрока, чей герой был убит 
                                    #   (не заполнено, если был убит не герой)
                    "unit": "npc_dota_hero_viper" # тип убитого юнита
                }, 
                ...
            ],

            # список событий: покупка предметов
            "purchase_log": [
                {
                    "time": -73,     # игровое время
                                     #   точка отсчета игрового времени (ноль) начинается через
                                     #   несколько минут после фактического начала матча, поэтому
                                     #   время некоторых событий может быть отрицательным
                    "item_id": 44    # купленный предмер (расшифровка в dictionaries/items.csv)
                }, 
                ...
            ]

            # список событий: выкуп героя из таверны
            "buyback_log": [
                {"time": 2507},
                ...
            ],

            # список событий: установка героем "наблюдателей", позволяющих команде 
            # следить за чатью игрового поля на некотором расстоянии от точки установки
            "obs_log": [
                {
                    "time": 1711,    # игровое время установки
                    "xy": [111, 130] # координаты игрового поля
                }, 
                ...
            ],
            "sen_log": [], # аналогично полю obs_log, другой тип "наблюдателя"

        },
        ...
    ],
    
    # отсчеты игрового времени, в которые вычисляются значения временных рядов
    "times": [0, 60, 120, 180, ...],

    # ключевые события игры
    "objectives": [
        {
            "time": 198,           # время события
            "type": "firstblood",  # тип события
            "player1": 6,          # параметры события, могут содержать
            "player2": 1           #   индексы игроков (player), 
                                   #   номер команды (team, 0 — Radiant, 1 — Dire)
        }, 
        {
            "time": 765, 
            "type": "tower_kill", 
            "player": 7, 
            "team": 1
        }, 
        ...
    ]
    
    # итог матча (отсутствует в тестовых матчах)
    "finish": {
        "duration": 2980,             # длительность в секундах
        "radiant_win": false,         # true, если победила команда Radiant
        "tower_status_radiant": 0,    # состояние башен у команд к концу игры
        "tower_status_dire": 1972,    #   (см. описание битовой маски)
        "barracks_status_dire": 63,   # состояние бараков у команд к концу игры
        "barracks_status_radiant": 0  #   (см. описание битовой маски)
    }
}
```

#### Описание полей состояния башен и бараков

Состояние башен к концу игры задается целым числом, закодировано в битах:

```
┌─┬─┬─┬─┬─────────────────────── Not used.
│ │ │ │ │ ┌───────────────────── Ancient Bottom
│ │ │ │ │ │ ┌─────────────────── Ancient Top
│ │ │ │ │ │ │ ┌───────────────── Bottom Tier 3
│ │ │ │ │ │ │ │ ┌─────────────── Bottom Tier 2
│ │ │ │ │ │ │ │ │ ┌───────────── Bottom Tier 1
│ │ │ │ │ │ │ │ │ │ ┌─────────── Middle Tier 3
│ │ │ │ │ │ │ │ │ │ │ ┌───────── Middle Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ ┌─────── Middle Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ ┌───── Top Tier 3
│ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─── Top Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─ Top Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
```

Состояние бараков к концу игры закодировано в битах целого числа:

```
┌─┬───────────── Not used.
│ │ ┌─────────── Bottom Ranged
│ │ │ ┌───────── Bottom Melee
│ │ │ │ ┌─────── Middle Ranged
│ │ │ │ │ ┌───── Middle Melee
│ │ │ │ │ │ ┌─── Top Ranged
│ │ │ │ │ │ │ ┌─ Top Melee
│ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0
```

## Извлечение признаков

Скрипт extract_features.py производит извлечение признаков из известной информации о матче за первые 5 игровых минут, составляет из них таблицу. Таблица поможет вам быстрее сформировать матрицу объект-признак, вектор ответов и начать применять методы машинного обучения для решения поставленной задачи.

Признаки, представленные в таблице `features.csv`, по мнению экспертов в предметной области являются наиболее важными для решения задачи предсказания победы команды. Тем не менее, не обязательно использовать эти признаки в исходном виде для применения методов машинного обучения — вы можете сделать новые признаки из имеющихся. Более того, признаки в файле `features.csv` содержат не всю информацию, известную про матч за первые 5 игровых минут. Вы можете использовать скрипт `extract_features.py` как пример и добавлять свои признаки для улучшения качества предсказания.

#### Пример чтения файла с признаками

In [1]:
import pandas as pd
features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

## Метрика качества

В качестве метрики качества мы будем использовать площадь под ROC-кривой (AUC-ROC). Обратите внимание, что AUC-ROC — это метрика качества для алгоритма, выдающего оценки принадлежности первому классу. Оба алгоритма, которые будут использоваться в проекте — градиентный бустинг, и логистическая регрессия — умеют выдавать такие оценки. Для этого нужно получать предсказания с помощью функции predict_proba. Она возвращает два столбца: первый содержит оценки принадлежности нулевому классу, второй — первому классу. Вам нужны значения из второго столбца:
```python
pred = clf.predict_proba(X_test)[:, 1]
```

## Руководство по решению

Вам необходимо провести описанные ниже два этапа исследования, написать по результатам каждого этапа небольшой отчет (ниже указаны вопросы, ответы на которые должны содержаться в отчете), и предоставить для ревью данный отчет и код, с помощью которого вы выполнили задание.

**Обратите внимание:** высокое качество работы на кросс-валидации (близкое к 100%) — это в первую очередь повод задуматься о том, правильно ли вы обучаете модель. Возможно, вы заглядываете в будущее или настраиваетесь на неправильном наборе признаков.

### Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

   
##### Измерение времени работы кода
```python
import time
import datetime

start_time = datetime.datetime.now()

time.sleep(3) # вместо этой строчки разместить замеряемый код

print 'Time elapsed:', datetime.datetime.now() - start_time
```

###### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [1]:
import pandas as pd
features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
X = features.iloc[:,:-6]
y = features.iloc[:,-6:]

In [4]:
X

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0


###### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
X.count()

start_time                  97230
lobby_type                  97230
r1_hero                     97230
r1_level                    97230
r1_xp                       97230
                            ...  
dire_tpscroll_count         97230
dire_boots_count            97230
dire_ward_observer_count    97230
dire_ward_sentry_count      97230
dire_first_ward_time        95404
Length: 102, dtype: int64

In [6]:
# фичи с пропусками в данных
for i in range(len(X.count())):
    if X.count()[i] != 97230:
        print(X.count().index[i], ':', X.count()[i])

first_blood_time : 77677
first_blood_team : 77677
first_blood_player1 : 77677
first_blood_player2 : 53243
radiant_bottle_time : 81539
radiant_courier_time : 96538
radiant_flying_courier_time : 69751
radiant_first_ward_time : 95394
dire_bottle_time : 81087
dire_courier_time : 96554
dire_flying_courier_time : 71132
dire_first_ward_time : 95404


12 фичей имеют пропуски в данных.  
Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение.  
Признаки для каждой команды (префиксы radiant_ и dire_): время первого приобретения командой предмета ... - если пропуск, то этот предмет не был приобретен командой

###### 3. Замените пропуски на нули с помощью функции fillna(). 
На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [7]:
X = X.fillna(0)

In [8]:
# проверка на заполненность пропусков
# фичи с пропусками в данных
for i in range(len(X.count())):
    if X.count()[i] != 97230:
        print(X.count().index[i], ':', X.count()[i])
print('all full')

all full


###### 4. Какой столбец содержит целевую переменную? Запишите его название.

In [9]:
y.columns[1]

'radiant_win'

In [10]:
y1 = y.iloc[:,1]

###### 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". 
- Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества.  
- Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30).   

Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [12]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
import time
import datetime

In [13]:
cv = KFold(n_splits=5, shuffle=True, random_state=42) # разбиение выборки

In [16]:
n_est = [10, 20, 30, 50]

for i in n_est:
    start_time = datetime.datetime.now()
    
    clf = GradientBoostingClassifier(n_estimators=i, random_state=241)
    q = np.mean(cross_val_score(clf, X, y1, cv=cv, scoring='roc_auc')) # выдает 5 значений, которые усредняем
    print(i, ':', q)
    
    print('Time elapsed:', datetime.datetime.now() - start_time)

10 : 0.6648506879750012
Time elapsed: 0:00:38.673380
20 : 0.6824618768044435
Time elapsed: 0:01:16.559598
30 : 0.6900064710388155
Time elapsed: 0:01:54.773767
50 : 0.6974943609466162
Time elapsed: 0:03:10.689417


Классификаторы настраивались долго. Для 30 деревьев это заняло 00:01:54.773767 времени. При этом получилось качество **69%**.   
При дальнейшем увеличении деревьев (с 30 до 50, например) качество возрастает незначительно, а время работы алгоритма сильно увеличивается.  
Если мы не боремся за каждую тысячную в качестве, то на 30 деревьях стоит остановиться и не использовать больше, так как это долго по времени

###### 6. Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

   - Использовать для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество — например, половину объектов. Подмножество лучше всего брать случайным, а не формировать его из первых m объектов.
   - Упростить модель — например, уменьшить глубину деревьев в градиентом бустинге (max_depth).
   - Попробовать линейные методы оценки

### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

##### Код для формирования "мешка слов" по героям
```python
# N — количество различных героев в выборке
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
```

**Важно:** не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()
X_fit = scaler.fit(X)
X_std = X_fit.transform(X)

###### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. 
Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
from sklearn.linear_model import LogisticRegression

In [34]:
cv = KFold(n_splits=5, shuffle=True, random_state=42) # разбиение выборки

In [36]:
c_param = [0.01, 0.1, 1.0, 10, 100]

for i in c_param:
    start_time = datetime.datetime.now()
    
    clf = LogisticRegression(C=i, penalty='l2')
    q = np.mean(cross_val_score(clf, X_std, y1, cv=cv, scoring='roc_auc')) # выдает 5 значений, которые усредняем
    print(i, ':', q)
    
    print('Time elapsed:', datetime.datetime.now() - start_time)

0.01 : 0.7165498862352037
Time elapsed: 0:00:03.739160
0.1 : 0.7165269451581027
Time elapsed: 0:00:03.802533
1.0 : 0.7165221087939807
Time elapsed: 0:00:03.883274
10 : 0.7165218906414321
Time elapsed: 0:00:04.174781
100 : 0.7165218101258131
Time elapsed: 0:00:03.725035


Лучший параметр С = 0.01, так как при нем время работы минимальное, а качество максимальное. Качество при нем **71,6549%**.  
У бустинга качество было хуже, скорее всего из-за того, что здесь учитывается масштабирование признаков и данные имеют линейную зависимость.  
Логистическая регрессия работает быстрее бустинга, так как ее алгоритм проще

###### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [18]:
cat_col = X[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]

In [19]:
X = X.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [20]:
X_fit = scaler.fit(X)
X_std = X_fit.transform(X)

In [21]:
c_param = [0.01, 0.1, 1.0, 10, 100]

for i in c_param:
    start_time = datetime.datetime.now()
    
    clf = LogisticRegression(C=i, penalty='l2')
    q = np.mean(cross_val_score(clf, X_std, y1, cv=cv, scoring='roc_auc')) # выдает 5 значений, которые усредняем
    print(i, ':', q)
    
    print('Time elapsed:', datetime.datetime.now() - start_time)

0.01 : 0.7165592000076536
Time elapsed: 0:00:03.486298
0.1 : 0.7165338144534041
Time elapsed: 0:00:03.562602
1.0 : 0.7165303443778955
Time elapsed: 0:00:03.735291
10 : 0.7165305328738455
Time elapsed: 0:00:03.531032
100 : 0.7165304163651
Time elapsed: 0:00:03.454786


Качество стало выше, но незначительно - **71,6559%**, С = 0.01. Скорее всего это из-за того, что убранные факторы были незначимы в регрессии, поэтому они не вносили большой вклад в предсказание. При их выкидывании предсказание особо не поменялось

###### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [22]:
hero_id = set()
for i in range(1, cat_col.shape[1]):
    for j in cat_col.iloc[:,i].unique():
        hero_id.add(j)

In [23]:
max(hero_id)

112

###### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. 
Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [24]:
# N — количество различных героев в выборке
N = max(hero_id)
X_pick = np.zeros((cat_col.iloc[:,1:].shape[0], N))

for i, match_id in enumerate(cat_col.iloc[:,1:].index):
    for p in range(5):
        X_pick[i, cat_col[['r%d_hero' % (p+1)]].loc[match_id]-1] = 1
        X_pick[i, cat_col[['d%d_hero' % (p+1)]].loc[match_id]-1] = -1
    print(len(cat_col.iloc[:,1:].index), ':', i, ':', match_id)

97230 : 0 : 0
97230 : 1 : 1
97230 : 2 : 2
97230 : 3 : 3
97230 : 4 : 4
97230 : 5 : 5
97230 : 6 : 8
97230 : 7 : 9
97230 : 8 : 11
97230 : 9 : 12
97230 : 10 : 14
97230 : 11 : 15
97230 : 12 : 17
97230 : 13 : 20
97230 : 14 : 21
97230 : 15 : 22
97230 : 16 : 23
97230 : 17 : 25
97230 : 18 : 26
97230 : 19 : 27
97230 : 20 : 28
97230 : 21 : 29
97230 : 22 : 30
97230 : 23 : 31
97230 : 24 : 32
97230 : 25 : 34
97230 : 26 : 35
97230 : 27 : 36
97230 : 28 : 38
97230 : 29 : 39
97230 : 30 : 40
97230 : 31 : 43
97230 : 32 : 44
97230 : 33 : 45
97230 : 34 : 46
97230 : 35 : 47
97230 : 36 : 48
97230 : 37 : 49
97230 : 38 : 50
97230 : 39 : 51
97230 : 40 : 52
97230 : 41 : 53
97230 : 42 : 54
97230 : 43 : 56
97230 : 44 : 57
97230 : 45 : 58
97230 : 46 : 59
97230 : 47 : 61
97230 : 48 : 65
97230 : 49 : 66
97230 : 50 : 67
97230 : 51 : 68
97230 : 52 : 69
97230 : 53 : 70
97230 : 54 : 73
97230 : 55 : 74
97230 : 56 : 75
97230 : 57 : 76
97230 : 58 : 77
97230 : 59 : 78
97230 : 60 : 79
97230 : 61 : 80
97230 : 62 : 81
97230 : 63

97230 : 473 : 565
97230 : 474 : 566
97230 : 475 : 567
97230 : 476 : 568
97230 : 477 : 570
97230 : 478 : 571
97230 : 479 : 572
97230 : 480 : 573
97230 : 481 : 574
97230 : 482 : 577
97230 : 483 : 578
97230 : 484 : 579
97230 : 485 : 580
97230 : 486 : 581
97230 : 487 : 583
97230 : 488 : 584
97230 : 489 : 587
97230 : 490 : 588
97230 : 491 : 589
97230 : 492 : 591
97230 : 493 : 593
97230 : 494 : 594
97230 : 495 : 595
97230 : 496 : 596
97230 : 497 : 597
97230 : 498 : 602
97230 : 499 : 604
97230 : 500 : 605
97230 : 501 : 606
97230 : 502 : 607
97230 : 503 : 608
97230 : 504 : 609
97230 : 505 : 610
97230 : 506 : 611
97230 : 507 : 612
97230 : 508 : 613
97230 : 509 : 614
97230 : 510 : 615
97230 : 511 : 616
97230 : 512 : 618
97230 : 513 : 619
97230 : 514 : 620
97230 : 515 : 621
97230 : 516 : 622
97230 : 517 : 623
97230 : 518 : 624
97230 : 519 : 625
97230 : 520 : 626
97230 : 521 : 627
97230 : 522 : 629
97230 : 523 : 630
97230 : 524 : 631
97230 : 525 : 632
97230 : 526 : 633
97230 : 527 : 636
97230 : 52

97230 : 931 : 1112
97230 : 932 : 1114
97230 : 933 : 1115
97230 : 934 : 1116
97230 : 935 : 1117
97230 : 936 : 1118
97230 : 937 : 1120
97230 : 938 : 1121
97230 : 939 : 1122
97230 : 940 : 1123
97230 : 941 : 1124
97230 : 942 : 1125
97230 : 943 : 1126
97230 : 944 : 1128
97230 : 945 : 1129
97230 : 946 : 1130
97230 : 947 : 1131
97230 : 948 : 1132
97230 : 949 : 1133
97230 : 950 : 1134
97230 : 951 : 1135
97230 : 952 : 1136
97230 : 953 : 1137
97230 : 954 : 1138
97230 : 955 : 1139
97230 : 956 : 1140
97230 : 957 : 1141
97230 : 958 : 1143
97230 : 959 : 1144
97230 : 960 : 1145
97230 : 961 : 1146
97230 : 962 : 1147
97230 : 963 : 1149
97230 : 964 : 1150
97230 : 965 : 1151
97230 : 966 : 1152
97230 : 967 : 1153
97230 : 968 : 1154
97230 : 969 : 1155
97230 : 970 : 1156
97230 : 971 : 1159
97230 : 972 : 1160
97230 : 973 : 1161
97230 : 974 : 1162
97230 : 975 : 1164
97230 : 976 : 1165
97230 : 977 : 1166
97230 : 978 : 1169
97230 : 979 : 1170
97230 : 980 : 1171
97230 : 981 : 1172
97230 : 982 : 1173
97230 : 983 

97230 : 1351 : 1607
97230 : 1352 : 1608
97230 : 1353 : 1609
97230 : 1354 : 1610
97230 : 1355 : 1611
97230 : 1356 : 1612
97230 : 1357 : 1613
97230 : 1358 : 1614
97230 : 1359 : 1616
97230 : 1360 : 1617
97230 : 1361 : 1618
97230 : 1362 : 1619
97230 : 1363 : 1620
97230 : 1364 : 1622
97230 : 1365 : 1624
97230 : 1366 : 1625
97230 : 1367 : 1626
97230 : 1368 : 1627
97230 : 1369 : 1628
97230 : 1370 : 1629
97230 : 1371 : 1631
97230 : 1372 : 1632
97230 : 1373 : 1633
97230 : 1374 : 1634
97230 : 1375 : 1635
97230 : 1376 : 1637
97230 : 1377 : 1638
97230 : 1378 : 1639
97230 : 1379 : 1640
97230 : 1380 : 1641
97230 : 1381 : 1643
97230 : 1382 : 1644
97230 : 1383 : 1645
97230 : 1384 : 1646
97230 : 1385 : 1647
97230 : 1386 : 1648
97230 : 1387 : 1649
97230 : 1388 : 1650
97230 : 1389 : 1651
97230 : 1390 : 1652
97230 : 1391 : 1653
97230 : 1392 : 1655
97230 : 1393 : 1656
97230 : 1394 : 1658
97230 : 1395 : 1659
97230 : 1396 : 1660
97230 : 1397 : 1661
97230 : 1398 : 1662
97230 : 1399 : 1663
97230 : 1400 : 1664


97230 : 1763 : 2095
97230 : 1764 : 2097
97230 : 1765 : 2098
97230 : 1766 : 2099
97230 : 1767 : 2100
97230 : 1768 : 2101
97230 : 1769 : 2102
97230 : 1770 : 2103
97230 : 1771 : 2106
97230 : 1772 : 2107
97230 : 1773 : 2108
97230 : 1774 : 2109
97230 : 1775 : 2111
97230 : 1776 : 2113
97230 : 1777 : 2114
97230 : 1778 : 2116
97230 : 1779 : 2117
97230 : 1780 : 2118
97230 : 1781 : 2119
97230 : 1782 : 2122
97230 : 1783 : 2123
97230 : 1784 : 2124
97230 : 1785 : 2126
97230 : 1786 : 2127
97230 : 1787 : 2130
97230 : 1788 : 2131
97230 : 1789 : 2132
97230 : 1790 : 2133
97230 : 1791 : 2134
97230 : 1792 : 2135
97230 : 1793 : 2136
97230 : 1794 : 2137
97230 : 1795 : 2139
97230 : 1796 : 2140
97230 : 1797 : 2141
97230 : 1798 : 2142
97230 : 1799 : 2143
97230 : 1800 : 2144
97230 : 1801 : 2145
97230 : 1802 : 2146
97230 : 1803 : 2147
97230 : 1804 : 2148
97230 : 1805 : 2149
97230 : 1806 : 2150
97230 : 1807 : 2151
97230 : 1808 : 2152
97230 : 1809 : 2153
97230 : 1810 : 2154
97230 : 1811 : 2155
97230 : 1812 : 2156


...

97230 : 95725 : 112620
97230 : 95726 : 112621
97230 : 95727 : 112622
97230 : 95728 : 112623
97230 : 95729 : 112625
97230 : 95730 : 112626
97230 : 95731 : 112627
97230 : 95732 : 112628
97230 : 95733 : 112630
97230 : 95734 : 112631
97230 : 95735 : 112633
97230 : 95736 : 112635
97230 : 95737 : 112636
97230 : 95738 : 112637
97230 : 95739 : 112638
97230 : 95740 : 112639
97230 : 95741 : 112640
97230 : 95742 : 112641
97230 : 95743 : 112643
97230 : 95744 : 112644
97230 : 95745 : 112645
97230 : 95746 : 112646
97230 : 95747 : 112648
97230 : 95748 : 112649
97230 : 95749 : 112650
97230 : 95750 : 112652
97230 : 95751 : 112653
97230 : 95752 : 112654
97230 : 95753 : 112655
97230 : 95754 : 112656
97230 : 95755 : 112657
97230 : 95756 : 112658
97230 : 95757 : 112659
97230 : 95758 : 112660
97230 : 95759 : 112661
97230 : 95760 : 112662
97230 : 95761 : 112663
97230 : 95762 : 112664
97230 : 95763 : 112666
97230 : 95764 : 112667
97230 : 95765 : 112668
97230 : 95766 : 112669
97230 : 95767 : 112670
97230 : 957

97230 : 96084 : 113059
97230 : 96085 : 113060
97230 : 96086 : 113061
97230 : 96087 : 113062
97230 : 96088 : 113063
97230 : 96089 : 113065
97230 : 96090 : 113066
97230 : 96091 : 113067
97230 : 96092 : 113068
97230 : 96093 : 113069
97230 : 96094 : 113070
97230 : 96095 : 113071
97230 : 96096 : 113072
97230 : 96097 : 113073
97230 : 96098 : 113074
97230 : 96099 : 113075
97230 : 96100 : 113076
97230 : 96101 : 113077
97230 : 96102 : 113078
97230 : 96103 : 113079
97230 : 96104 : 113080
97230 : 96105 : 113082
97230 : 96106 : 113083
97230 : 96107 : 113084
97230 : 96108 : 113085
97230 : 96109 : 113086
97230 : 96110 : 113087
97230 : 96111 : 113088
97230 : 96112 : 113089
97230 : 96113 : 113090
97230 : 96114 : 113091
97230 : 96115 : 113092
97230 : 96116 : 113093
97230 : 96117 : 113094
97230 : 96118 : 113096
97230 : 96119 : 113097
97230 : 96120 : 113098
97230 : 96121 : 113101
97230 : 96122 : 113103
97230 : 96123 : 113104
97230 : 96124 : 113106
97230 : 96125 : 113107
97230 : 96126 : 113108
97230 : 961

97230 : 96456 : 113502
97230 : 96457 : 113504
97230 : 96458 : 113505
97230 : 96459 : 113506
97230 : 96460 : 113507
97230 : 96461 : 113508
97230 : 96462 : 113510
97230 : 96463 : 113511
97230 : 96464 : 113512
97230 : 96465 : 113513
97230 : 96466 : 113514
97230 : 96467 : 113515
97230 : 96468 : 113517
97230 : 96469 : 113518
97230 : 96470 : 113519
97230 : 96471 : 113521
97230 : 96472 : 113522
97230 : 96473 : 113523
97230 : 96474 : 113524
97230 : 96475 : 113525
97230 : 96476 : 113526
97230 : 96477 : 113527
97230 : 96478 : 113528
97230 : 96479 : 113529
97230 : 96480 : 113530
97230 : 96481 : 113532
97230 : 96482 : 113533
97230 : 96483 : 113534
97230 : 96484 : 113535
97230 : 96485 : 113536
97230 : 96486 : 113538
97230 : 96487 : 113539
97230 : 96488 : 113541
97230 : 96489 : 113542
97230 : 96490 : 113543
97230 : 96491 : 113544
97230 : 96492 : 113545
97230 : 96493 : 113546
97230 : 96494 : 113547
97230 : 96495 : 113548
97230 : 96496 : 113549
97230 : 96497 : 113550
97230 : 96498 : 113551
97230 : 964

97230 : 96832 : 113946
97230 : 96833 : 113947
97230 : 96834 : 113948
97230 : 96835 : 113949
97230 : 96836 : 113950
97230 : 96837 : 113951
97230 : 96838 : 113952
97230 : 96839 : 113953
97230 : 96840 : 113954
97230 : 96841 : 113956
97230 : 96842 : 113958
97230 : 96843 : 113959
97230 : 96844 : 113960
97230 : 96845 : 113961
97230 : 96846 : 113962
97230 : 96847 : 113963
97230 : 96848 : 113964
97230 : 96849 : 113965
97230 : 96850 : 113966
97230 : 96851 : 113967
97230 : 96852 : 113968
97230 : 96853 : 113969
97230 : 96854 : 113970
97230 : 96855 : 113971
97230 : 96856 : 113972
97230 : 96857 : 113973
97230 : 96858 : 113974
97230 : 96859 : 113975
97230 : 96860 : 113976
97230 : 96861 : 113977
97230 : 96862 : 113978
97230 : 96863 : 113980
97230 : 96864 : 113982
97230 : 96865 : 113983
97230 : 96866 : 113984
97230 : 96867 : 113985
97230 : 96868 : 113986
97230 : 96869 : 113987
97230 : 96870 : 113989
97230 : 96871 : 113990
97230 : 96872 : 113991
97230 : 96873 : 113992
97230 : 96874 : 113993
97230 : 968

97230 : 97190 : 114362
97230 : 97191 : 114363
97230 : 97192 : 114364
97230 : 97193 : 114365
97230 : 97194 : 114366
97230 : 97195 : 114367
97230 : 97196 : 114368
97230 : 97197 : 114370
97230 : 97198 : 114371
97230 : 97199 : 114372
97230 : 97200 : 114373
97230 : 97201 : 114374
97230 : 97202 : 114375
97230 : 97203 : 114376
97230 : 97204 : 114379
97230 : 97205 : 114380
97230 : 97206 : 114381
97230 : 97207 : 114382
97230 : 97208 : 114383
97230 : 97209 : 114384
97230 : 97210 : 114385
97230 : 97211 : 114386
97230 : 97212 : 114387
97230 : 97213 : 114388
97230 : 97214 : 114389
97230 : 97215 : 114390
97230 : 97216 : 114391
97230 : 97217 : 114392
97230 : 97218 : 114394
97230 : 97219 : 114395
97230 : 97220 : 114396
97230 : 97221 : 114397
97230 : 97222 : 114399
97230 : 97223 : 114400
97230 : 97224 : 114401
97230 : 97225 : 114402
97230 : 97226 : 114403
97230 : 97227 : 114404
97230 : 97228 : 114405
97230 : 97229 : 114406


In [26]:
X_pick.shape

(97230, 112)

In [49]:
X_hero = pd.DataFrame(X_pick, 
                      index=cat_col.iloc[:,1:].index, 
                      columns=[i for i in range(1, 113)])

In [50]:
X_new = X.join(X_hero)

###### 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [51]:
X_fit = scaler.fit(X_new)
X_std = X_fit.transform(X_new)

In [52]:
c_param = [0.01, 0.1, 1.0, 10, 100]

for i in c_param:
    start_time = datetime.datetime.now()
    
    clf = LogisticRegression(C=i, penalty='l2')
    q = np.mean(cross_val_score(clf, X_std, y1, cv=cv, scoring='roc_auc')) # выдает 5 значений, которые усредняем
    print(i, ':', q)
    
    print('Time elapsed:', datetime.datetime.now() - start_time)

0.01 : 0.7519639455861148
Time elapsed: 0:00:07.064785
0.1 : 0.7519296127153968
Time elapsed: 0:00:07.901916
1.0 : 0.7519244690811582
Time elapsed: 0:00:07.921390
10 : 0.7519241577673921
Time elapsed: 0:00:07.666429
100 : 0.7519241366014189
Time elapsed: 0:00:08.169332


Качество получилось **75,1963%** при С = 0.01. Качество увеличилось довольно сильно. Это можно объяснить добавлением большего количества фичей в выборку, которые вносят вклад в целевую переменную

###### 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).
Лучшей моделью оказалась последняя, ее и будем использовать

In [55]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [56]:
X_test = features_test.fillna(0)

In [57]:
cat_col = X_test[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
X_test = X_test.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [58]:
# N — количество различных героев в выборке
N = max(hero_id)
X_pick = np.zeros((cat_col.iloc[:,1:].shape[0], N))

for i, match_id in enumerate(cat_col.iloc[:,1:].index):
    for p in range(5):
        X_pick[i, cat_col[['r%d_hero' % (p+1)]].loc[match_id]-1] = 1
        X_pick[i, cat_col[['d%d_hero' % (p+1)]].loc[match_id]-1] = -1
    print(len(cat_col.iloc[:,1:].index), ':', i, ':', match_id)

17177 : 0 : 6
17177 : 1 : 7
17177 : 2 : 10
17177 : 3 : 13
17177 : 4 : 16
17177 : 5 : 18
17177 : 6 : 19
17177 : 7 : 24
17177 : 8 : 33
17177 : 9 : 37
17177 : 10 : 41
17177 : 11 : 42
17177 : 12 : 55
17177 : 13 : 60
17177 : 14 : 62
17177 : 15 : 63
17177 : 16 : 64
17177 : 17 : 71
17177 : 18 : 72
17177 : 19 : 83
17177 : 20 : 85
17177 : 21 : 89
17177 : 22 : 92
17177 : 23 : 100
17177 : 24 : 102
17177 : 25 : 108
17177 : 26 : 111
17177 : 27 : 126
17177 : 28 : 130
17177 : 29 : 140
17177 : 30 : 141
17177 : 31 : 152
17177 : 32 : 153
17177 : 33 : 159
17177 : 34 : 160
17177 : 35 : 163
17177 : 36 : 178
17177 : 37 : 179
17177 : 38 : 181
17177 : 39 : 196
17177 : 40 : 206
17177 : 41 : 209
17177 : 42 : 216
17177 : 43 : 228
17177 : 44 : 230
17177 : 45 : 242
17177 : 46 : 245
17177 : 47 : 261
17177 : 48 : 274
17177 : 49 : 282
17177 : 50 : 289
17177 : 51 : 296
17177 : 52 : 297
17177 : 53 : 302
17177 : 54 : 304
17177 : 55 : 313
17177 : 56 : 316
17177 : 57 : 317
17177 : 58 : 344
17177 : 59 : 348
17177 : 60 : 35

17177 : 452 : 2844
17177 : 453 : 2846
17177 : 454 : 2847
17177 : 455 : 2857
17177 : 456 : 2861
17177 : 457 : 2888
17177 : 458 : 2903
17177 : 459 : 2904
17177 : 460 : 2907
17177 : 461 : 2909
17177 : 462 : 2910
17177 : 463 : 2915
17177 : 464 : 2918
17177 : 465 : 2920
17177 : 466 : 2926
17177 : 467 : 2929
17177 : 468 : 2947
17177 : 469 : 2950
17177 : 470 : 2952
17177 : 471 : 2953
17177 : 472 : 2956
17177 : 473 : 2961
17177 : 474 : 2962
17177 : 475 : 2967
17177 : 476 : 2973
17177 : 477 : 2974
17177 : 478 : 2981
17177 : 479 : 3001
17177 : 480 : 3006
17177 : 481 : 3007
17177 : 482 : 3008
17177 : 483 : 3014
17177 : 484 : 3022
17177 : 485 : 3032
17177 : 486 : 3033
17177 : 487 : 3043
17177 : 488 : 3048
17177 : 489 : 3061
17177 : 490 : 3063
17177 : 491 : 3087
17177 : 492 : 3088
17177 : 493 : 3096
17177 : 494 : 3102
17177 : 495 : 3111
17177 : 496 : 3119
17177 : 497 : 3122
17177 : 498 : 3126
17177 : 499 : 3131
17177 : 500 : 3132
17177 : 501 : 3145
17177 : 502 : 3154
17177 : 503 : 3178
17177 : 504 

17177 : 906 : 5918
17177 : 907 : 5929
17177 : 908 : 5935
17177 : 909 : 5938
17177 : 910 : 5940
17177 : 911 : 5942
17177 : 912 : 5950
17177 : 913 : 5963
17177 : 914 : 5968
17177 : 915 : 5969
17177 : 916 : 5970
17177 : 917 : 5978
17177 : 918 : 5987
17177 : 919 : 5990
17177 : 920 : 5997
17177 : 921 : 5998
17177 : 922 : 6001
17177 : 923 : 6004
17177 : 924 : 6005
17177 : 925 : 6011
17177 : 926 : 6013
17177 : 927 : 6029
17177 : 928 : 6043
17177 : 929 : 6045
17177 : 930 : 6049
17177 : 931 : 6051
17177 : 932 : 6064
17177 : 933 : 6066
17177 : 934 : 6071
17177 : 935 : 6078
17177 : 936 : 6085
17177 : 937 : 6086
17177 : 938 : 6095
17177 : 939 : 6103
17177 : 940 : 6105
17177 : 941 : 6106
17177 : 942 : 6107
17177 : 943 : 6111
17177 : 944 : 6114
17177 : 945 : 6118
17177 : 946 : 6139
17177 : 947 : 6140
17177 : 948 : 6147
17177 : 949 : 6153
17177 : 950 : 6156
17177 : 951 : 6158
17177 : 952 : 6163
17177 : 953 : 6165
17177 : 954 : 6176
17177 : 955 : 6178
17177 : 956 : 6182
17177 : 957 : 6199
17177 : 958 

17177 : 1326 : 8687
17177 : 1327 : 8710
17177 : 1328 : 8711
17177 : 1329 : 8712
17177 : 1330 : 8715
17177 : 1331 : 8718
17177 : 1332 : 8719
17177 : 1333 : 8720
17177 : 1334 : 8727
17177 : 1335 : 8732
17177 : 1336 : 8738
17177 : 1337 : 8739
17177 : 1338 : 8748
17177 : 1339 : 8753
17177 : 1340 : 8754
17177 : 1341 : 8756
17177 : 1342 : 8759
17177 : 1343 : 8768
17177 : 1344 : 8780
17177 : 1345 : 8787
17177 : 1346 : 8789
17177 : 1347 : 8794
17177 : 1348 : 8802
17177 : 1349 : 8816
17177 : 1350 : 8831
17177 : 1351 : 8833
17177 : 1352 : 8846
17177 : 1353 : 8849
17177 : 1354 : 8850
17177 : 1355 : 8864
17177 : 1356 : 8872
17177 : 1357 : 8875
17177 : 1358 : 8900
17177 : 1359 : 8911
17177 : 1360 : 8923
17177 : 1361 : 8939
17177 : 1362 : 8940
17177 : 1363 : 8951
17177 : 1364 : 8954
17177 : 1365 : 8958
17177 : 1366 : 8960
17177 : 1367 : 8965
17177 : 1368 : 8978
17177 : 1369 : 8980
17177 : 1370 : 8989
17177 : 1371 : 8991
17177 : 1372 : 8997
17177 : 1373 : 9001
17177 : 1374 : 9009
17177 : 1375 : 9016


17177 : 1729 : 11249
17177 : 1730 : 11258
17177 : 1731 : 11267
17177 : 1732 : 11278
17177 : 1733 : 11284
17177 : 1734 : 11289
17177 : 1735 : 11309
17177 : 1736 : 11311
17177 : 1737 : 11321
17177 : 1738 : 11335
17177 : 1739 : 11344
17177 : 1740 : 11350
17177 : 1741 : 11356
17177 : 1742 : 11357
17177 : 1743 : 11359
17177 : 1744 : 11361
17177 : 1745 : 11362
17177 : 1746 : 11374
17177 : 1747 : 11376
17177 : 1748 : 11379
17177 : 1749 : 11385
17177 : 1750 : 11391
17177 : 1751 : 11402
17177 : 1752 : 11414
17177 : 1753 : 11422
17177 : 1754 : 11428
17177 : 1755 : 11440
17177 : 1756 : 11450
17177 : 1757 : 11469
17177 : 1758 : 11474
17177 : 1759 : 11476
17177 : 1760 : 11484
17177 : 1761 : 11488
17177 : 1762 : 11489
17177 : 1763 : 11531
17177 : 1764 : 11532
17177 : 1765 : 11550
17177 : 1766 : 11555
17177 : 1767 : 11562
17177 : 1768 : 11568
17177 : 1769 : 11570
17177 : 1770 : 11573
17177 : 1771 : 11575
17177 : 1772 : 11589
17177 : 1773 : 11590
17177 : 1774 : 11605
17177 : 1775 : 11607
17177 : 1776 

...

17177 : 15565 : 103762
17177 : 15566 : 103772
17177 : 15567 : 103777
17177 : 15568 : 103781
17177 : 15569 : 103799
17177 : 15570 : 103801
17177 : 15571 : 103803
17177 : 15572 : 103816
17177 : 15573 : 103821
17177 : 15574 : 103825
17177 : 15575 : 103826
17177 : 15576 : 103832
17177 : 15577 : 103838
17177 : 15578 : 103839
17177 : 15579 : 103842
17177 : 15580 : 103850
17177 : 15581 : 103852
17177 : 15582 : 103855
17177 : 15583 : 103874
17177 : 15584 : 103877
17177 : 15585 : 103894
17177 : 15586 : 103910
17177 : 15587 : 103921
17177 : 15588 : 103928
17177 : 15589 : 103933
17177 : 15590 : 103936
17177 : 15591 : 103938
17177 : 15592 : 103949
17177 : 15593 : 103973
17177 : 15594 : 103975
17177 : 15595 : 103977
17177 : 15596 : 103982
17177 : 15597 : 103984
17177 : 15598 : 103994
17177 : 15599 : 104001
17177 : 15600 : 104003
17177 : 15601 : 104007
17177 : 15602 : 104009
17177 : 15603 : 104020
17177 : 15604 : 104022
17177 : 15605 : 104035
17177 : 15606 : 104036
17177 : 15607 : 104041
17177 : 156

17177 : 15928 : 106129
17177 : 15929 : 106135
17177 : 15930 : 106137
17177 : 15931 : 106155
17177 : 15932 : 106161
17177 : 15933 : 106175
17177 : 15934 : 106191
17177 : 15935 : 106205
17177 : 15936 : 106210
17177 : 15937 : 106211
17177 : 15938 : 106212
17177 : 15939 : 106217
17177 : 15940 : 106224
17177 : 15941 : 106229
17177 : 15942 : 106240
17177 : 15943 : 106243
17177 : 15944 : 106247
17177 : 15945 : 106255
17177 : 15946 : 106265
17177 : 15947 : 106275
17177 : 15948 : 106280
17177 : 15949 : 106283
17177 : 15950 : 106293
17177 : 15951 : 106300
17177 : 15952 : 106304
17177 : 15953 : 106305
17177 : 15954 : 106307
17177 : 15955 : 106309
17177 : 15956 : 106317
17177 : 15957 : 106320
17177 : 15958 : 106321
17177 : 15959 : 106334
17177 : 15960 : 106342
17177 : 15961 : 106348
17177 : 15962 : 106350
17177 : 15963 : 106351
17177 : 15964 : 106352
17177 : 15965 : 106360
17177 : 15966 : 106374
17177 : 15967 : 106380
17177 : 15968 : 106395
17177 : 15969 : 106399
17177 : 15970 : 106404
17177 : 159

17177 : 16300 : 108657
17177 : 16301 : 108665
17177 : 16302 : 108677
17177 : 16303 : 108679
17177 : 16304 : 108681
17177 : 16305 : 108685
17177 : 16306 : 108727
17177 : 16307 : 108736
17177 : 16308 : 108747
17177 : 16309 : 108759
17177 : 16310 : 108761
17177 : 16311 : 108766
17177 : 16312 : 108767
17177 : 16313 : 108768
17177 : 16314 : 108774
17177 : 16315 : 108775
17177 : 16316 : 108776
17177 : 16317 : 108785
17177 : 16318 : 108797
17177 : 16319 : 108800
17177 : 16320 : 108814
17177 : 16321 : 108817
17177 : 16322 : 108843
17177 : 16323 : 108844
17177 : 16324 : 108847
17177 : 16325 : 108848
17177 : 16326 : 108855
17177 : 16327 : 108858
17177 : 16328 : 108862
17177 : 16329 : 108863
17177 : 16330 : 108870
17177 : 16331 : 108881
17177 : 16332 : 108891
17177 : 16333 : 108894
17177 : 16334 : 108897
17177 : 16335 : 108898
17177 : 16336 : 108907
17177 : 16337 : 108916
17177 : 16338 : 108937
17177 : 16339 : 108945
17177 : 16340 : 108948
17177 : 16341 : 108950
17177 : 16342 : 108957
17177 : 163

17177 : 16663 : 110996
17177 : 16664 : 111008
17177 : 16665 : 111011
17177 : 16666 : 111023
17177 : 16667 : 111028
17177 : 16668 : 111032
17177 : 16669 : 111045
17177 : 16670 : 111051
17177 : 16671 : 111065
17177 : 16672 : 111080
17177 : 16673 : 111094
17177 : 16674 : 111140
17177 : 16675 : 111143
17177 : 16676 : 111144
17177 : 16677 : 111147
17177 : 16678 : 111163
17177 : 16679 : 111166
17177 : 16680 : 111171
17177 : 16681 : 111175
17177 : 16682 : 111179
17177 : 16683 : 111194
17177 : 16684 : 111199
17177 : 16685 : 111207
17177 : 16686 : 111211
17177 : 16687 : 111218
17177 : 16688 : 111224
17177 : 16689 : 111242
17177 : 16690 : 111243
17177 : 16691 : 111245
17177 : 16692 : 111247
17177 : 16693 : 111249
17177 : 16694 : 111255
17177 : 16695 : 111257
17177 : 16696 : 111258
17177 : 16697 : 111260
17177 : 16698 : 111265
17177 : 16699 : 111270
17177 : 16700 : 111274
17177 : 16701 : 111276
17177 : 16702 : 111282
17177 : 16703 : 111288
17177 : 16704 : 111297
17177 : 16705 : 111305
17177 : 167

17177 : 17043 : 113495
17177 : 17044 : 113496
17177 : 17045 : 113501
17177 : 17046 : 113503
17177 : 17047 : 113509
17177 : 17048 : 113516
17177 : 17049 : 113520
17177 : 17050 : 113531
17177 : 17051 : 113537
17177 : 17052 : 113540
17177 : 17053 : 113553
17177 : 17054 : 113555
17177 : 17055 : 113567
17177 : 17056 : 113573
17177 : 17057 : 113577
17177 : 17058 : 113584
17177 : 17059 : 113598
17177 : 17060 : 113625
17177 : 17061 : 113631
17177 : 17062 : 113632
17177 : 17063 : 113637
17177 : 17064 : 113655
17177 : 17065 : 113668
17177 : 17066 : 113672
17177 : 17067 : 113673
17177 : 17068 : 113676
17177 : 17069 : 113679
17177 : 17070 : 113681
17177 : 17071 : 113688
17177 : 17072 : 113691
17177 : 17073 : 113693
17177 : 17074 : 113700
17177 : 17075 : 113702
17177 : 17076 : 113704
17177 : 17077 : 113722
17177 : 17078 : 113727
17177 : 17079 : 113730
17177 : 17080 : 113733
17177 : 17081 : 113739
17177 : 17082 : 113740
17177 : 17083 : 113746
17177 : 17084 : 113749
17177 : 17085 : 113762
17177 : 170

In [59]:
X_hero = pd.DataFrame(X_pick, 
                      index=cat_col.iloc[:,1:].index, 
                      columns=[i for i in range(1, 113)])

In [60]:
X_test_new = X_test.join(X_hero)

In [61]:
X_test = X_fit.transform(X_test_new)

clf = LogisticRegression(C=0.01, penalty='l2').fit(X_std, y1)
pred = clf.predict_proba(X_test)[:, 1]

In [66]:
pred

array([0.8227909 , 0.75216683, 0.18906701, ..., 0.2379355 , 0.6283895 ,
       0.42762756])

In [67]:
for i in pred:
    if i < 0 or i > 1:
        print('smth wrong')
print('it`s ok')

it`s ok


1. Какое качество получилось у логистической регрессии над всеми исходными признаками? **71,6549%** Как оно соотносится с качеством градиентного бустинга? **оно выше на 2%** Чем вы можете объяснить эту разницу? **данные имеют линейную зависимость** Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? **быстрее**
2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? **71,6559%** Чем вы можете объяснить это изменение? **эти признаки незначимы**
3. Сколько различных идентификаторов героев существует в данной игре? **в train выборке 108, а всего 112**
4. Какое получилось качество при добавлении "мешка слов" по героям? **75,1963%** Улучшилось ли оно по сравнению с предыдущим вариантом? **да** Чем вы можете это объяснить? **факторы значимы в модели, но только в преобразованном виде - необходимо кодирование категориальных данных**
5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [69]:
print('min:', pred.min())
print('max:', pred.max())

min: 0.00850599069841976
max: 0.9962795503882248


## Проверка финальной модели

После того как вы провели все эксперименты и выбрали лучшую модель, можете проверить ее качество на тестовых матчах. Выборка тестовых матчей собрана в файле `matches_test.jsonlines.bz2`. В отличие от основного набора матчей, в тестовых матчах есть только та информация, которая известна на момент первых 5 игровых минут, результат матча — неизвестен. Таблица признаков для тестовых матчей — `features_test.csv`.

Для всех матчей из тестового набора предскажите вероятность победы Radiant, запишите предсказания в CSV файл с колонками `match_id` (идентификатор матча) и `radiant_win` — предсказанная вероятность. Файл с предсказаниями должен выглядеть примерно следующим образом:

```
match_id,radiant_win
1,0.51997370502
4,0.51997370502
15,0.51997370502
...
```

Отправьте решение на Kaggle в соревнование: Dota 2: Win Probability Prediction.

Ссылка на соревнование: [Dota 2: Win Probability Prediction](https://kaggle.com/join/coursera_ml_dota2_contest)

In [82]:
y_pred = pd.DataFrame(pred, features_test.index, columns=['radiant_win'])

In [84]:
y_pred.head()

,radiant_win
match_id,
6,0.822791
7,0.752167
10,0.189067
13,0.857639
16,0.243964


In [85]:
y_pred.to_csv('winner_game', sep=',')

### Что еще попробовать?

Разумеется, можно попробовать еще очень много разных идей, которые помогут вам получить еще более высокий результат на kaggle. Вот лишь несколько возможных вариантов:
1. Про каждого из игроков есть достаточно много показателей: максимальный опыт, число смертей и т.д. (см. список выше). Можно попробовать просуммировать или усредних их, получив агрегированные показатели для всей команды.
2. В сырых данных (файл matches.jsonlines.bz2) содержится очень много информации, которую мы пока не использовали. Вы можете, например, составить "мешки слов" для покупок различных предметов (то есть кодировать информацию о том, сколько раз каждая команда покупала тот или иной предмет). Обратите внимание, что при этом вы можете получить слишком большое количество признаков, для которых может иметь смысл сделать понижение размерности с помощью метода главных компонент.
3. Можно сформировать признаки про изменения способностей героев в течение матча (ability_upgrades).
4. В этом задании используются только градиентный бустинг и логистическая регрессия — но ведь мы изучали и другие модели! Можно попробовать метод k ближайших соседей, SVM, случайный лес и так далее.